In [2]:
# Connecting to the db
import lib.handle_db as dbh

# read and write csv files
import lib.handle_csv as csv_rw

# date functions
from datetime import datetime, date, timedelta

#CR libraries
from crossref.restful import Works, Etiquette

# search for UKCH Awards in CR record
def award_in_crossref(aw):
    ukch_wks =[]
    not_revised = []
    for wk in aw:      
        awd_list = []
        if 'funder' in wk.keys():
            for fdr in wk['funder']:
                if 'award' in fdr.keys():
                   awds = 0
                   for awd in fdr['award']:
                        if awd in ['EP/R026939/1', 'EP/R026815/1', 'EP/R026645/1', 'EP/R027129/1', 'EP/M013219/1',
                                  'EP/K014706/2', 'EP/K014668/1', 'EP/K014854/1', 'EP/K014714/1']:
                            awd_list.append(awd)
                            #print (fdr)
                            break
        else:
            not_revised.append(wk)
        if len(awd_list) > 0:
            ukch_wks.append(wk)
    return ukch_wks, not_revised

def collect_keys(aw, wk_keys):
    for wk in aw:
        these_keys = wk.keys()
        for a_key in these_keys:
            if a_key in wk_keys:
                wk_keys[a_key] += 1
            else:
                wk_keys[a_key] = 1
    return wk_keys

def collect_fdr_keys(aw, wk_keys):
    for wk in aw:
        if 'funder' in wk.keys():
            for fdr in wk['funder']:
                these_keys = fdr.keys()
                for a_key in these_keys:
                    if a_key in wk_keys:
                        wk_keys[a_key] += 1
                    else:
                        wk_keys[a_key] = 1
    return wk_keys

def collect_awds(aw, wk_keys):
    for wk in aw:
        if 'funder' in wk.keys():
            for fdr in wk['funder']:
                if 'award' in fdr.keys():
                   awds = 0
                   for awd in fdr['award']:
                        if awd in wk_keys:
                            wk_keys[awd] += 1
                        else:
                            wk_keys[awd] = 1
    return wk_keys

# search for UKCH Affiliation in CR record
def affi_in_crossref(aw):
    ukch_wks = []
    for wk in aw:
        ukch_affiliation = False
        if 'author' in wk.keys():
            for autr in wk['author']:
                if 'affiliation' in autr.keys():
                    for affi in autr['affiliation']:
                        if "UK Catalysis Hub" in affi['name']:
                            ukch_affiliation = True
                            break
                    if ukch_affiliation:
                        ukch_wks.append(wk)
                        break
    return  ukch_wks

In [4]:
my_etiquette = Etiquette('UK Catalysis Hub Catalysis Data Infrastructure', 'Prototype 1', 'https://ukcatalysishub.co.uk/core/', 'nievadelahidalgaa@cardiff.ac.uk')


works = Works(etiquette=my_etiquette)

start_date = date(2022, 4, 2)
end_date = date(2022, 6, 1)

pubs_with_award = []
skiped_works =[]
wk_keys = {}
fd_keys = {}
awds_lst = {}
while end_date < datetime.now().date():
    end_date = start_date + timedelta(days=7)
    print ("From:", str(start_date), "to",  str(end_date))
    # works with from_published_date and until_published_date 
    # next test with from_deposit_date and until_deposit_date
    # Valid filters for this route are: alternative_id, archive, article_number, assertion, assertion-group, 
    #    award.funder, award.number, category-name, clinical-trial-number, container-title, content-domain,
    #    directory, doi, from-accepted-date, from-created-date, from-deposit-date, from-event-end-date,
    #    from-event-start-date, from-index-date, from-issued-date, from-online-pub-date, from-posted-date,
    #    from-print-pub-date, from-pub-date, from-update-date, full-text.application, full-text.type, 
    #    full-text.version, funder, funder-doi-asserted-by, group-title, has-abstract, has-affiliation,
    #    has-archive, has-assertion, has-authenticated-orcid, has-award, has-clinical-trial-number,
    #    has-content-domain, has-domain-restriction, has-event, has-full-text, has-funder, has-funder-doi,
    #    has-license, has-orcid, has-references, has-relation, has-update, has-update-policy, is-update, 
    #    isbn, issn, license.delay, license.url, license.version, location, member, orcid, prefix,
    #    relation.object, relation.object-type, relation.type, type, type-name, until-accepted-date,
    #    until-created-date, until-deposit-date, until-event-end-date, until-event-start-date,
    #    until-index-date, until-issued-date, until-online-pub-date, until-posted-date,
    #    until-print-pub-date, until-pub-date, until-update-date, update-type, updates

    print(works.filter(has_funder='true').filter(from_deposit_date=str(start_date)).filter(until_deposit_date=str(end_date)).url)
    pub_w_grant = works.filter(has_funder='true').filter(from_deposit_date=str(start_date)).filter(until_deposit_date=str(end_date))
    
    aw, _  = award_in_crossref(pub_w_grant)
    ukch_wks = []
    
    for wk in aw:     
        
        awd_list = [] 
        for fdr in wk['funder']:
            if 'award' in fdr.keys():
               awds = 0
               for awd in fdr['award']:
                   if awd in ['EP/R026939/1', 'EP/R026815/1', 'EP/R026645/1', 'EP/R027129/1', 'EP/M013219/1',
                              'EP/K014706/2', 'EP/K014668/1', 'EP/K014854/1', 'EP/K014714/1']:
                       awd_list.append(awd)
        if len(awd_list) > 0:
            ukch_wks.append(wk)
    
    foud_pubs = {}
    if len(ukch_wks) > 0:
        foud_pubs = {}
        for wk in ukch_wks:
            art_authors = ""
            if 'author' in wk.keys() :
                for autr in wk['author']:
                    if art_authors == "":
                        art_authors = autr['family']+", " + (", "+ autr ['given'] if 'given' in autr.keys() else "" )
                    else:
                        art_authors += ", " + autr['family']+ (", "+ autr ['given'] if 'given' in autr.keys() else "" )
            fund_award = ""
            for fdr in wk['funder']:
                if 'award' in fdr.keys():
                  for awd in fdr['award']:
                       if awd in ['EP/R026939/1', 'EP/R026815/1', 'EP/R026645/1', 'EP/R027129/1', 'EP/M013219/1',
                                  'EP/K014706/2', 'EP/K014668/1', 'EP/K014854/1', 'EP/K014714/1']:
                            if fund_award  == "":
                                fund_award = awd
                            else:
                                fund_award += ", " +awd

            ol_year = 0
            pr_year = 0
            pub_year = 0
            if 'published-online' in wk.keys() and 'date-parts' in wk['published-online'].keys():
                ol_year = int(wk['published-online']['date-parts'][0][0])
            if 'published-print' in wk.keys() and 'date-parts' in wk['published-print'].keys():
                pr_year = int(wk['published-print']['date-parts'][0][0])
            if pr_year > 0 and ol_year > 0:
                if pr_year > ol_year:
                    pub_year = ol_year
                else:
                    pub_year = pr_year
            elif ol_year > 0:
                pub_year = ol_year
            elif pr_year > 0:
                pub_year = pr_year

            print(art_authors,"|",pub_year,"|",wk['title'][0],
                  "|", wk['DOI'],"|", fund_award)     
            this_pub = {}
            this_pub['authors'] = art_authors
            this_pub['year'] = pub_year
            this_pub['title'] = wk['title'][0]
            this_pub['DOI'] = wk['DOI']
            this_pub['awards'] = fund_award
            if not wk['DOI'] in foud_pubs:
                 foud_pubs[wk['DOI']]= this_pub

        
        # WRITE TO FILE
        if len(foud_pubs) > 0:
            csv_rw.write_csv_data(foud_pubs, 'cr_check_'+str(end_date)+'a.csv') 
        
    start_date = end_date + timedelta(days=1)

From: 2022-04-02 to 2022-04-09
https://api.crossref.org/works?filter=has-funder%3Atrue%2Cfrom-deposit-date%3A2022-04-02%2Cuntil-deposit-date%3A2022-04-09
Lam, , Francis Y. T., Wells, Jordann A. L., Ochiai, Tatsumi, Halliday, Connor J. V., McCabe, Karl N., Maron, Laurent, Arnold, Polly L. | 2022 | A Combined Experimental and Theoretical Investigation of Arene-Supported Actinide and Ytterbium Tetraphenolate Complexes | 10.1021/acs.inorgchem.1c03365 | EP/K014714/1
Celorrio, , V., Tiwari, D., Calvillo, L., Leach, A., Huang, H., Granozzi, G., Alonso, J.A., Aguadero, A., Pinacca, R.M., Russell, A.E., Fermin, D.J. | 2021 | Electrocatalytic Site Activity Enhancement via Orbital Overlap in A<sub>2</sub>MnRuO<sub>7</sub> (A = Dy<sup>3+</sup>, Ho<sup>3+</sup>, and Er<sup>3+</sup>) Pyrochlore Nanostructures | 10.1021/acsaem.0c02060 | EP/K014714/1
Cherevotan, , Arjun, Raj, Jithu, Dheer, Lakshay, Roy, Soumyabrata, Sarkar, Shreya, Das, Risov, Vinod, Chathakudath P., Xu, Shaojun, Wells, Peter, Waghmar

Sun, , Shuzhuang, Sun, Hongman, Guan, Shaoliang, Xu, Shaojun, Wu, Chunfei | 2022 | Integrated CO2 capture and methanation on Ru/CeO2-MgO combined materials: Morphology effect from CeO2 support | 10.1016/j.fuel.2022.123420 | EP/R026939/1
Kamran, , Muhammad, Davidson, Matthew G., de Vos, Sicco, Tsanaktsis, Vasilios, Yeniad, Bahar | 2022 | Synthesis and characterisation of polyamides based on 2,5-furandicarboxylic acid as a sustainable building block for engineering plastics | 10.1039/d2py00189f | EP/R027129/1
From: 2022-05-28 to 2022-06-04
https://api.crossref.org/works?filter=has-funder%3Atrue%2Cfrom-deposit-date%3A2022-05-28%2Cuntil-deposit-date%3A2022-06-04
Chapman, , Stephanie, Brookes, Catherine, Bowker, Michael, Gibson, Emma K., Wells, Peter P. | 2016 | Design and stabilisation of a high area iron molybdate surface for the selective oxidation of methanol to formaldehyde | 10.1039/c5fd00153f | EP/K014668/1, EP/K014714/1, EP/K014854/1
Di Carmine, , Graziano, Pesciaioli, Fabio, Wang, 

In [3]:
#end_date = date(2022, 3, 10)

foud_pubs = {'10.1039/d1gc04414a': {'authors': 'Burnett, , Joseph W. H., Li, Jianwei, McCue, Alan J., Kechagiopoulos, Panagiotis N., Howe, Russell F., Wang, Xiaodong',
  'year': 2022,
  'title': 'Directing the H2-driven selective regeneration of NADH via Sn-doped Pt/SiO2',
  'DOI': '10.1039/d1gc04414a',
  'awards': 'EP/R026645/1'},
 '10.1039/d1ee03523a': {'authors': 'Erans, , María, Sanz-Pérez, Eloy S., Hanak, Dawid P., Clulow, Zeynep, Reiner, David M., Mutch, Greg A.',
  'year': 2022,
  'title': 'Direct air capture: process technology, techno-economic and socio-political challenges',
  'DOI': '10.1039/d1ee03523a',
  'awards': 'EP/R027129/1'},
 '10.1016/j.jechem.2021.05.005': {'authors': 'Arrigo, , Rosa, Sasaki, Takeo, Callison, June, Gianolio, Diego, Schuster, Manfred Erwin',
  'year': 2022,
  'title': 'Monitoring dynamics of defects and single Fe atoms in N-functionalized few-layer graphene by in situ temperature programmed scanning transmission electron microscopy',
  'DOI': '10.1016/j.jechem.2021.05.005',
  'awards': 'EP/K014668/1, EP/K014714/1, EP/K014854/1'},
 '10.1039/d1cy02361f': {'authors': 'Minova, , Ivalina B., Bühl, Michael, Matam, Santhosh K., Catlow, C. Richard A., Frogley, Mark D., Cinque, Gianfelice, Wright, Paul A., Howe, Russell F.',
  'year': 2022,
  'title': 'Carbene-like reactivity of methoxy groups in a single crystal SAPO-34 MTO catalyst',
  'DOI': '10.1039/d1cy02361f',
  'awards': 'EP/K014668/1, EP/K014714/1, EP/K014854/1, EP/M013219/1'},
 '10.1039/d1cy02001c': {'authors': "Porter, , A. J., Botchway, C. H., Kwakye-Awuah, B., Hernandez-Tamargo, C., Matam, S. K., McHugh, S. L., Silverwood, I. P., de Leeuw, N. H., O'Malley, A. J.",
  'year': 2022,
  'title': 'Local and nanoscale methanol mobility in different H-FER catalysts',
  'DOI': '10.1039/d1cy02001c',
  'awards': 'EP/R026815/1, EP/R026645/1, EP/R026939/1, EP/R027129/1'}}

if len(foud_pubs) > 0:
    csv_rw.write_csv_data(foud_pubs, 'cr_check_'+str(end_date)+'a.csv') 

In [4]:
 'cr_check_'+str(end_date)+'.csv'

'cr_check_2022-04-06.csv'

In [5]:
wk

{'indexed': {'date-parts': [[2022, 3, 27]],
  'date-time': '2022-03-27T22:41:09Z',
  'timestamp': 1648420869112},
 'reference-count': 69,
 'publisher': 'American Chemical Society (ACS)',
 'issue': '6',
 'license': [{'start': {'date-parts': [[2022, 3, 3]],
    'date-time': '2022-03-03T00:00:00Z',
    'timestamp': 1646265600000},
   'content-version': 'stm-asf',
   'delay-in-days': 0,
   'URL': 'https://doi.org/10.15223/policy-029'},
  {'start': {'date-parts': [[2022, 3, 3]],
    'date-time': '2022-03-03T00:00:00Z',
    'timestamp': 1646265600000},
   'content-version': 'stm-asf',
   'delay-in-days': 0,
   'URL': 'https://doi.org/10.15223/policy-037'},
  {'start': {'date-parts': [[2022, 3, 3]],
    'date-time': '2022-03-03T00:00:00Z',
    'timestamp': 1646265600000},
   'content-version': 'stm-asf',
   'delay-in-days': 0,
   'URL': 'https://doi.org/10.15223/policy-045'},
  {'start': {'date-parts': [[2023, 3, 2]],
    'date-time': '2023-03-02T00:00:00Z',
    'timestamp': 1677715200000},
 

In [6]:
ukch_wks

[]